In [ ]:
pip install Skforecast

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster

# Modelling and Forecasting
# ==============================================================================
from lightgbm import LGBMRegressor

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

from joblib import dump, load

# Configuration
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [4]:
FORECAST_HORIZON = 90

def make_prediction(historical_values: pd.DataFrame,
                    forecast_horizon: int = FORECAST_HORIZON) -> pd.DataFrame:
    """Function creates oil production forecast for a single well
    fitting decline curve to previous data.
    :param historical_values: DataFrame containing parameters for one well
    :param forecast_horizon: Number of steps (days) in the forecast
    :returns: DataFrame containing forecast
    """
    historical_values['datetime'] = pd.to_datetime(historical_values['datetime'], format='%Y-%m-%d')
    historical_values[['Объем нефти', 'Объем жидкости']] = historical_values[['Объем нефти', 'Объем жидкости']].fillna(method = 'backfill')
    historical_values['Показатель обводненности'] = historical_values['Объем нефти']/historical_values['Объем жидкости']
    historical_values.drop(['Объем жидкости', 
                        'x', 
                        'y', 
                        'Активная мощность (ТМ)', 
                        'Давление буферное',
                        'Газовый фактор рабочий (ТМ)'
                       ], axis = 1, inplace = True)
    historical_values.drop_duplicates('datetime', inplace=True)
    historical_values = historical_values.set_index('datetime')
    historical_values = historical_values.asfreq('D')
    historical_values = historical_values.sort_index()
    historical_values = historical_values.fillna(method = 'backfill')
    historical_values['Дебит газа (ТМ)'] = historical_values['Дебит газа (ТМ)'].fillna(method = 'ffill')
    historical_values = historical_values.fillna(method = 'ffill')
    #Select exogenous variables
    exog_variables = ['Объем нефти',
                      'Давление линейное (ТМ)',
                      'Дебит газа попутного',
                      'Коэффициент мощности (ТМ)',
                      'Показатель обводненности']
    
    forecaster_exog = ForecasterAutoreg(
                regressor = LGBMRegressor(),
                lags = 30
                )
    
    date_range = pd.date_range(start='1992-04-11', freq='1D', periods=forecast_horizon)
    
    forecaster_exog.fit(historical_values['Объем нефти'])
    predictions_exog = forecaster_exog.predict(steps=forecast_horizon)
    forecast_exog = pd.DataFrame({'datetime': date_range, 'Объем нефти': predictions_exog})
    
    forecaster_exog.fit(historical_values['Давление линейное (ТМ)'])
    predictions_exog = forecaster_exog.predict(steps=forecast_horizon)
    forecast_exog['Давление линейное (ТМ)'] =  predictions_exog
    
    forecaster_exog.fit(historical_values['Дебит газа попутного'])
    predictions_exog = forecaster_exog.predict(steps=forecast_horizon)
    forecast_exog['Дебит газа попутного'] =  predictions_exog
    
    forecaster_exog.fit(historical_values['Коэффициент мощности (ТМ)'])
    predictions_exog = forecaster_exog.predict(steps=forecast_horizon)
    forecast_exog['Коэффициент мощности (ТМ)'] =  predictions_exog
    
    forecaster_exog.fit(historical_values['Показатель обводненности'])
    predictions_exog = forecaster_exog.predict(steps=forecast_horizon)
    forecast_exog['Показатель обводненности'] =  predictions_exog
    
    forecast_exog.drop('datetime', axis = 1, inplace = True)
    
    
    forecaster = ForecasterAutoreg(
                regressor = LGBMRegressor(),
                lags = 30
                )
    
    param_grid = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1]
    }
    
    # Lags used as predictors
    lags_grid = [24, 48, 72, [1, 2, 3, 23, 24, 25, 71, 72, 73]]


    results_grid = grid_search_forecaster(
        forecaster         = forecaster,
        y                  = historical_values.loc[:, 'Дебит нефти'], # Train and validation data
        exog               = historical_values.loc[:, exog_variables],
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = 60,
        refit              = False,
        metric             = 'mean_squared_error',
        initial_train_size = 75, # Model is trained with trainign data
        fixed_train_size   = False,
        return_best        = True,
        verbose            = False
        ) 
    
    predictions = forecaster.predict(steps = 90, exog = forecast_exog)
    forecast_df = pd.DataFrame({'datetime': date_range, 'forecast': predictions})
    return forecast_df

In [5]:
def process_data():
    """Function loads training data from file and
    iterates over unique wells in the dataset
    making forecast for each time series.
    """
    train_path = Path().cwd().parent / 'data' / 'train.csv'
    train_df = pd.read_csv(train_path)
    print(f'Loaded training data. Shape: {train_df.shape}')

    wells = list(train_df['Номер скважины'].unique())
    print(f'Number of unique wells: {len(wells)}')

    all_forecasts = []
    with tqdm(total=len(wells)) as pbar:
        for well in wells:
            print(f'Started processing well ID: {well}')
            well_df = train_df[train_df['Номер скважины'] == well]

            # Make prediction using Decline Curve Analysis
            forecats_df = make_prediction(well_df)
            forecats_df['Номер скважины'] = [well] * len(forecats_df)
            all_forecasts.append(forecats_df)

            pbar.update(1)
    all_forecasts = pd.concat(all_forecasts)
    print(f'Completed data processing. Forecast shape: {all_forecasts.shape}')
    print(f'Number of unique wells: {len(all_forecasts["Номер скважины"].unique())}')

    all_forecasts.to_csv('forecast.csv', index=False, encoding="utf-8")
    print('Saved forecast to "forecast.csv"')


In [6]:
process_data()

Loaded training data. Shape: (67136, 20)
Number of unique wells: 106


  0%|                                                                                          | 0/106 [00:00<?, ?it/s]

Started processing well ID: 0
Number of models compared: 48.


  1%|▊                                                                                 | 1/106 [00:08<15:06,  8.63s/it]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3 23 24 25 71 72 73] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 500}
  Backtesting metric: 97.69166734125677

Started processing well ID: 1
Number of models compared: 48.


...

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 500}
  Backtesting metric: 69.08014863145003

Started processing well ID: 104
Number of models compared: 48.


 99%|███████████████████████████████████████████████████████████████████████████████▏| 105/106 [16:58<00:09,  9.73s/it]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 500}
  Backtesting metric: 5.466364683657019

Started processing well ID: 105
Number of models compared: 48.


100%|████████████████████████████████████████████████████████████████████████████████| 106/106 [17:08<00:00,  9.71s/it]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 500}
  Backtesting metric: 10.255881211759881

Completed data processing. Forecast shape: (9540, 3)
Number of unique wells: 106
Saved forecast to "baseline_forecast_10.csv"
